In [1]:

import pandas as pd
import numpy as np
import gsw  # Gibbs-Seawater Oceanographic Toolbox
import os


# Process a list of files and append the bottom steric pressure to a table
def process_files_and_save(file_paths, output_csv):
    for file_path in file_paths:
        # Extract the file name (without extension) to use as an identifier
        file_name = os.path.basename(file_path).split('.')[0]
        print(file_name)

        # Define column widths and names based on file format
        colspecs = [(0, 8), (8, 16), (16, 25), (25, 33), (33, 41), (41, 49), (49, 57), (57, 65)]
        column_names = ['Pressure', 'Temperature', 'Salinity', 'Oxygen', 'Transmission/Nitrate', 'Chloropigments', 'Observations', 'Quality']

        # Read the data from the file
        ctd_data = pd.read_fwf(file_path, colspecs=colspecs, skiprows=6, names=column_names)

        # Calculate depth using the calculate_depth function
        ctd_data['Depth'] = calculate_depth(ctd_data['Pressure'], ctd_data['Temperature'], ctd_data['Salinity'])

        # Calculate steric pressure
        ctd_data['StericPressure'] = calc_sfp_steric_integrated(
            ctd_data['Temperature'].values, 
            ctd_data['Depth'].values, 
        )

        # Get the bottom steric pressure (last value in the StericPressure column)
        bottom_steric_pressure = ctd_data['StericPressure'].iloc[-1]

        # Create a DataFrame with the file name and bottom steric pressure
        result_df = pd.DataFrame({'FileName': [file_name], 'BottomStericPressure': [bottom_steric_pressure]})

        # Append the result to the output CSV file
        if os.path.exists(output_csv):
            result_df.to_csv(output_csv, mode='a', header=False, index=False)
        else:
            result_df.to_csv(output_csv, mode='w', header=True, index=False)

# Function to generate the file paths for folders 318 to 325 and files a0201 to a0220
def generate_file_paths(base_dir, start_folder=326, end_folder=334, start_file=201, end_file=222):
    file_paths = []
    
    for folder_num in range(start_folder, end_folder + 1):
        folder_name = f'hot-{folder_num}'
        folder_path = os.path.join(base_dir, folder_name)
        
        # Check if the folder exists
        if os.path.exists(folder_path):
            for file_num in range(start_file, end_file + 1):
                file_name = f'h{folder_num}a0{file_num}.ctd'
                file_path = os.path.join(folder_path, file_name)
                
                # Only add the file if it exists
                if os.path.exists(file_path):
                    file_paths.append(file_path)
                    #print({file_path})
                else:
                    print(f"File not found: {file_path}, skipping.")
        else:
            print(f"Folder not found: {folder_path}, skipping.")
    
    return file_paths

# Base directory where the folders are located
base_dir = r"F:\aloha\ALOHA Cabled Observatory Database\ACO\hot"

# Output CSV file path
output_csv = r"F:\aloha\ALOHA Cabled Observatory Database\ACO\hot\hot_raw_2021.csv"

# Generate the file paths for folders 318 to 325 and files a0201 to a0220
file_paths = generate_file_paths(base_dir, start_folder=326, end_folder=334, start_file=201, end_file=222)

# Process the files and save the results
process_files_and_save(file_paths, output_csv)

print("Data saved to:", output_csv)

NameError: name 'os' is not defined